In [46]:
import pandas as pd
from matplotlib import pyplot as plt

data = pd.DataFrame(columns = ['country', 'year', 'pollution_co2'])
df = pd.read_csv('./assets/CO2_Emissions_1960-2018.csv')

# Pays gardés
kept_countries = ['France', 'Italy', 'Spain', 'Germany', 'Portugal', 'United Kingdom']
limit_to_kept_countries = False

df.head(5)

,Country Name,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Aruba,204.631696,208.837879,226.081890,214.785217,207.626699,185.213644,172.158729,210.819017,194.917536,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Africa Eastern and Southern,0.906060,0.922474,0.930816,0.940570,0.996033,1.047280,1.033908,1.052204,1.079727,...,1.021954,1.048876,1.005338,1.021646,1.031833,1.041145,0.987393,0.971016,0.959978,0.933541
2,Afghanistan,0.046057,0.053589,0.073721,0.074161,0.086174,0.101285,0.107399,0.123409,0.115142,...,0.211306,0.297065,0.407074,0.335351,0.263716,0.234037,0.232176,0.208857,0.203328,0.200151
3,Africa Western and Central,0.090880,0.095283,0.096612,0.112376,0.133258,0.184803,0.193676,0.189305,0.143989,...,0.426770,0.472819,0.497023,0.490867,0.504655,0.507671,0.480743,0.472959,0.476438,0.515544
4,Angola,0.100835,0.082204,0.210533,0.202739,0.213562,0.205891,0.268937,0.172096,0.289702,...,1.205902,1.221515,1.216317,1.204799,1.261542,1.285365,1.260921,1.227703,1.034317,0.887380


In [47]:
# Récupération de la liste des pays
countries = df['Country Name'].unique().tolist()
print(countries)
df.set_index('Country Name', inplace=True)

['Aruba', 'Africa Eastern and Southern', 'Afghanistan', 'Africa Western and Central', 'Angola', 'Albania', 'Andorra', 'Arab World', 'United Arab Emirates', 'Argentina', 'Armenia', 'American Samoa', 'Antigua and Barbuda', 'Australia', 'Austria', 'Azerbaijan', 'Burundi', 'Belgium', 'Benin', 'Burkina Faso', 'Bangladesh', 'Bulgaria', 'Bahrain', 'Bahamas, The', 'Bosnia and Herzegovina', 'Belarus', 'Belize', 'Bermuda', 'Bolivia', 'Brazil', 'Barbados', 'Brunei Darussalam', 'Bhutan', 'Botswana', 'Central African Republic', 'Canada', 'Central Europe and the Baltics', 'Switzerland', 'Channel Islands', 'Chile', 'China', "Cote d'Ivoire", 'Cameroon', 'Congo, Dem. Rep.', 'Congo, Rep.', 'Colombia', 'Comoros', 'Cabo Verde', 'Costa Rica', 'Caribbean small states', 'Cuba', 'Curacao', 'Cayman Islands', 'Cyprus', 'Czech Republic', 'Germany', 'Djibouti', 'Dominica', 'Denmark', 'Dominican Republic', 'Algeria', 'East Asia & Pacific (excluding high income)', 'Early-demographic dividend', 'East Asia & Pacific'

In [48]:
# Récupération de la liste des années
columns = df.columns
years = columns[1:].tolist()
print(years)

['1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018']


In [49]:
# Création nouveau dataframe organisé par pays puis années
data_list = []

for country in countries:
    #values = df[df['Country Name'] == country]
    for year in years:
        value = df.loc[country, year]
        #print(value)
        data_list.append({'country': country, 'year': year, 'pollution_co2': value})
data = pd.DataFrame(data_list, columns=['country', 'year', 'pollution_co2'])
data.head(5)

,country,year,pollution_co2
0,Aruba,1961,208.837879
1,Aruba,1962,226.081890
2,Aruba,1963,214.785217
3,Aruba,1964,207.626699
4,Aruba,1965,185.213644


In [50]:
data.dtypes

country           object
year              object
pollution_co2    float64
dtype: object

In [51]:
data.shape

(15428, 3)

In [52]:
# Suppression des valeurs inutiles si nécessaires
nb_na = data['pollution_co2'].isna().sum()
print("Nombre de valeurs invalides ou nulls: " + str(nb_na))
if nb_na > 0:
    data.dropna(subset=['pollution_co2'], inplace=True)
data.shape

Nombre de valeurs invalides ou nulls: 2289


(13139, 3)

In [53]:
data.to_csv('./output/CO2_Emissions_sortedByYearAndCountry.csv', index=False)

In [54]:
# Retirer les valeurs des pays différents de cette liste et exporter le fichier
if limit_to_kept_countries:
    data = data[data.country.isin(kept_countries)]
data.to_csv('./output/CO2_Emissions_sortedByYearAndCountry.csv', index=False)


In [55]:
# Charger le fichier World Energy Consumption
wec = pd.read_csv('./assets/World Energy Consumption.csv')
if limit_to_kept_countries:
    wec = wec[wec.country.isin(kept_countries)]
wec.to_csv('./output/World Energy Consumption (limited).csv', index=False)

In [56]:
# Changement de types
data['year'] = data['year'].astype(int)
wec['year'] = wec['year'].astype(int)

wec['country'] = wec['country'].astype(str)
data['country'] = data['country'].astype(str)

# Fusion des valeurs de Co2
wec = pd.merge(wec, data, on=["country", "year"])
            
# Supprimer la colonne 'population'
# wec = wec.drop('population', axis=1, errors='ignore')
# wec = wec.drop('iso_code', axis=1, errors='ignore')
# wec = wec.drop('gdp', axis=1, errors='ignore')

wec = wec[wec['year'] >= 1960]

# Valeurs enregistrés
wec.to_csv('./output/World Energy Consumption (limited).csv', index=False)

In [57]:
#Dataset Taxes sur les énergies

In [58]:
# Charger le dataset des taxes sur les énergies
tax = pd.read_csv('./assets/Tax_Energy.csv')
tax.head(5)

,ObjectId,Country,ISO2,ISO3,Indicator,Source,CTS Code,CTS Name,CTS Full Descriptor,Unit,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,1,Albania,AL,ALB,Environmental Taxes,Organisation for Economic Co-operation and Dev...,ECGTE,Environmental Taxes,"Environment, Climate Change, Government Policy...",Domestic Currency,...,NaN,NaN,NaN,4.399314e+10,4.781379e+10,4.754858e+10,5.114559e+10,5.341565e+10,NaN,NaN
1,2,Albania,AL,ALB,Environmental Taxes,Organisation for Economic Co-operation and Dev...,ECGTE,Environmental Taxes,"Environment, Climate Change, Government Policy...",Percent of GDP,...,NaN,NaN,NaN,3.067206e+00,3.247163e+00,3.066373e+00,3.124865e+00,3.157133e+00,NaN,NaN
2,3,Albania,AL,ALB,Taxes on Energy (including fuel for transport),Organisation for Economic Co-operation and Dev...,ECGTEN,Taxes on Energy (Including Fuel for Transport),"Environment, Climate Change, Government Policy...",Domestic Currency,...,NaN,NaN,NaN,3.774111e+10,4.094562e+10,4.040004e+10,4.352182e+10,4.516530e+10,NaN,NaN
3,4,Albania,AL,ALB,Taxes on Energy (including fuel for transport),Organisation for Economic Co-operation and Dev...,ECGTEN,Taxes on Energy (Including Fuel for Transport),"Environment, Climate Change, Government Policy...",Percent of GDP,...,NaN,NaN,NaN,2.631314e+00,2.780726e+00,2.605369e+00,2.659072e+00,2.669496e+00,NaN,NaN
4,5,Albania,AL,ALB,Taxes on Pollution,Organisation for Economic Co-operation and Dev...,ECGTEP,Taxes on Pollution,"Environment, Climate Change, Government Policy...",Domestic Currency,...,NaN,NaN,NaN,1.782069e+09,1.879970e+09,1.941324e+09,2.226251e+09,2.625011e+09,NaN,NaN


In [59]:
tax_country = tax[tax["Country"] == "Andorra, Principality of"]
print(tax_country)

    ObjectId                   Country ISO2 ISO3  \
10        11  Andorra, Principality of   AD  AND   
11        12  Andorra, Principality of   AD  AND   
12        13  Andorra, Principality of   AD  AND   
13        14  Andorra, Principality of   AD  AND   
14        15  Andorra, Principality of   AD  AND   

                                            Indicator  \
10                                Environmental Taxes   
11     Taxes on Energy (including fuel for transport)   
12                                 Taxes on Pollution   
13                                 Taxes on Resources   
14  Taxes on Transport (excluding fuel for transport)   

                                               Source CTS Code  \
10  Organisation for Economic Co-operation and Dev...    ECGTE   
11  Organisation for Economic Co-operation and Dev...   ECGTEN   
12  Organisation for Economic Co-operation and Dev...   ECGTEP   
13  Organisation for Economic Co-operation and Dev...   ECGTER   
14  Organisati

In [60]:
# Compter le nombre de valeurs "NaN" dans les colonnes
nb_nan_colTax = tax.isna().sum()

print("Nombre de NaN dans les colonnes :", nb_nan_colTax)

# Compter le nombre de valeurs "NaN" dans le dataset
nb_nan_dataset = tax.isna().sum().sum()

print("Nombre de NaN dans le dataset :", nb_nan_dataset)

Nombre de NaN dans les colonnes : ObjectId                 0
Country                  0
ISO2                    10
ISO3                     8
Indicator                0
Source                   0
CTS Code                 0
CTS Name                 0
CTS Full Descriptor      0
Unit                     0
1995                   495
1996                   491
1997                   491
1998                   483
1999                   481
2000                   431
2001                   415
2002                   411
2003                   403
2004                   403
2005                   369
2006                   359
2007                   351
2008                   321
2009                   321
2010                   295
2011                   303
2012                   307
2013                   309
2014                   305
2015                    44
2016                    46
2017                    50
2018                    68
2019                   100
2020                 

In [61]:
# Regrouper les années dans une seule colonne
tax = pd.melt(tax, id_vars=["ObjectId", "Country", "ISO2", "ISO3", "Indicator", "Source", "CTS Code", "CTS Name", "CTS Full Descriptor", "Unit"], 
                    var_name="Année", value_name="Taxe")
tax.head(5)

,ObjectId,Country,ISO2,ISO3,Indicator,Source,CTS Code,CTS Name,CTS Full Descriptor,Unit,Année,Taxe
0,1,Albania,AL,ALB,Environmental Taxes,Organisation for Economic Co-operation and Dev...,ECGTE,Environmental Taxes,"Environment, Climate Change, Government Policy...",Domestic Currency,1995,NaN
1,2,Albania,AL,ALB,Environmental Taxes,Organisation for Economic Co-operation and Dev...,ECGTE,Environmental Taxes,"Environment, Climate Change, Government Policy...",Percent of GDP,1995,NaN
2,3,Albania,AL,ALB,Taxes on Energy (including fuel for transport),Organisation for Economic Co-operation and Dev...,ECGTEN,Taxes on Energy (Including Fuel for Transport),"Environment, Climate Change, Government Policy...",Domestic Currency,1995,NaN
3,4,Albania,AL,ALB,Taxes on Energy (including fuel for transport),Organisation for Economic Co-operation and Dev...,ECGTEN,Taxes on Energy (Including Fuel for Transport),"Environment, Climate Change, Government Policy...",Percent of GDP,1995,NaN
4,5,Albania,AL,ALB,Taxes on Pollution,Organisation for Economic Co-operation and Dev...,ECGTEP,Taxes on Pollution,"Environment, Climate Change, Government Policy...",Domestic Currency,1995,NaN


In [62]:
# Concaténer les valeurs de "CTS Code" et "CTS Name" avec un séparateur "-"
tax["CTS Name"] = tax["CTS Code"] + " - " + tax["CTS Name"]

# Supprimer la colonne "CTS Code" puisque ses valeurs ont été concaténées avec "CTS Name"
# Supprimer les colonnes inutiles "ObjectId", "ISO2", "Indicator", "Source", "CTS Full Descriptor"
tax.drop(columns=["CTS Code", "ObjectId", "ISO2", "Indicator", "Source", "CTS Full Descriptor"], inplace=True)

tax.head(5)

,Country,ISO3,CTS Name,Unit,Année,Taxe
0,Albania,ALB,ECGTE - Environmental Taxes,Domestic Currency,1995,NaN
1,Albania,ALB,ECGTE - Environmental Taxes,Percent of GDP,1995,NaN
2,Albania,ALB,ECGTEN - Taxes on Energy (Including Fuel for T...,Domestic Currency,1995,NaN
3,Albania,ALB,ECGTEN - Taxes on Energy (Including Fuel for T...,Percent of GDP,1995,NaN
4,Albania,ALB,ECGTEP - Taxes on Pollution,Domestic Currency,1995,NaN


In [63]:
tax.columns

Index(['Country', 'ISO3', 'CTS Name', 'Unit', 'Année', 'Taxe'], dtype='object')

In [64]:
tax.columns.dtype

dtype('O')

In [65]:
tax.dtypes

Country      object
ISO3         object
CTS Name     object
Unit         object
Année        object
Taxe        float64
dtype: object

In [66]:
tax.shape

(33453, 6)

In [67]:
# Vérifier les valeurs nulles dans le DataFrame
null_values = tax.isnull()

# Vérifier si le DataFrame est vide
is_empty = tax.empty

# Afficher les valeurs nulles
print("Valeurs nulles dans le DataFrame :")
print(null_values)

# Afficher si le DataFrame est vide
print("\nLe DataFrame est vide :", is_empty)

Valeurs nulles dans le DataFrame :
       Country   ISO3  CTS Name   Unit  Année  Taxe
0        False  False     False  False  False  True
1        False  False     False  False  False  True
2        False  False     False  False  False  True
3        False  False     False  False  False  True
4        False  False     False  False  False  True
...        ...    ...       ...    ...    ...   ...
33448    False  False     False  False  False  True
33449    False  False     False  False  False  True
33450    False  False     False  False  False  True
33451    False  False     False  False  False  True
33452    False  False     False  False  False  True

[33453 rows x 6 columns]

Le DataFrame est vide : False


In [68]:
# Vérifier s'il y a des valeurs "nan" ou "null" dans la colonne 'Taxe'
nan_colTaxe = tax['Taxe'].isna().any()

print("Valeurs nan ou null dans la colonne 'Taxe':", nan_colTaxe)

Valeurs nan ou null dans la colonne 'Taxe': True


In [69]:
# Compter le nombre de valeurs "NaN" dans la colonne 'Taxe'
nb_nan_colTax = tax['Taxe'].isna().sum()

print("Nombre de NaN dans la colonne 'Taxe':", nb_nan_colTax)

Nombre de NaN dans la colonne 'Taxe': 8823


In [70]:
# Filtrer les lignes avec des valeurs  puis les afficher
nan_rows = tax[tax.isna().any(axis=1)]

print("Lignes contenant des valeurs NaN :")
print(nan_rows)

Lignes contenant des valeurs NaN :
                  Country ISO3  \
0                 Albania  ALB   
1                 Albania  ALB   
2                 Albania  ALB   
3                 Albania  ALB   
4                 Albania  ALB   
...                   ...  ...   
33448  West Bank and Gaza  PSE   
33449  West Bank and Gaza  PSE   
33450  West Bank and Gaza  PSE   
33451  West Bank and Gaza  PSE   
33452  West Bank and Gaza  PSE   

                                                CTS Name               Unit  \
0                            ECGTE - Environmental Taxes  Domestic Currency   
1                            ECGTE - Environmental Taxes     Percent of GDP   
2      ECGTEN - Taxes on Energy (Including Fuel for T...  Domestic Currency   
3      ECGTEN - Taxes on Energy (Including Fuel for T...     Percent of GDP   
4                            ECGTEP - Taxes on Pollution  Domestic Currency   
...                                                  ...                ...   
334

In [71]:
# Compter le nombre de valeurs "NaN" dans les colonnes
nb_nan_colTax = tax.isna().sum()

print("Nombre de NaN dans les colonnes :", nb_nan_colTax)

# Compter le nombre de valeurs "NaN" dans le dataset
nb_nan_dataset = tax.isna().sum().sum()

print("Nombre de NaN dans le dataset :", nb_nan_dataset)

Nombre de NaN dans les colonnes : Country        0
ISO3         216
CTS Name       0
Unit           0
Année          0
Taxe        8823
dtype: int64
Nombre de NaN dans le dataset : 9039


In [72]:
print(tax.columns)

Index(['Country', 'ISO3', 'CTS Name', 'Unit', 'Année', 'Taxe'], dtype='object')


In [73]:
# Pivoter les données pour mettre les catégories des taxes de la colonne "CTS Name" en colonnes et pour mettre les unités des taxes de la colonne "Unit" en colonnes
taxE = tax.pivot_table(index=['Country', 'ISO3', 'Année'],
                               columns=['CTS Name', 'Unit'],
                               values='Taxe').reset_index()

taxE.head(5)

CTS Name  Country ISO3 Année ECGTE - Environmental Taxes                 \
Unit                                   Domestic Currency Percent of GDP   
0         Albania  ALB  2015                4.399314e+10       3.067206   
1         Albania  ALB  2016                4.781379e+10       3.247163   
2         Albania  ALB  2017                4.754858e+10       3.066373   
3         Albania  ALB  2018                5.114559e+10       3.124865   
4         Albania  ALB  2019                5.341565e+10       3.157133   

CTS Name ECGTEN - Taxes on Energy (Including Fuel for Transport)  \
Unit                                           Domestic Currency   
0                                              3.774111e+10        
1                                              4.094562e+10        
2                                              4.040004e+10        
3                                              4.352182e+10        
4                                              4.516530e+10        

CTS Name                ECGTEP - Taxes on Pollution                 \
Unit     Percent of GDP           Domestic Currency Percent of GDP   
0              2.631314                1.782069e+09       0.124246   
1              2.780726                1.879970e+09       0.127674   
2              2.605369                1.941324e+09       0.125195   
3              2.659072                2.226251e+09       0.136018   
4              2.669496                2.625011e+09       0.155151   

CTS Name ECGTER - Taxes on Resources                 \
Unit               Domestic Currency Percent of GDP   
0                        32546493.00       0.002269   
1                        52524339.00       0.003567   
2                        61861356.38       0.003989   
3                        56011991.00       0.003422   
4                        58777099.50       0.003474   

CTS Name ECGTET - Taxes on Transport (Excluding Fuel for Transport)  \
Unit                                              Domestic Currency   
0                                              4.437413e+09           
1                                              4.935684e+09           
2                                              5.145347e+09           
3                                              5.341506e+09           
4                                              5.566564e+09           

CTS Name                 
Unit     Percent of GDP  
0              0.309377  
1              0.335195  
2              0.331820  
3              0.326352  
4              0.329012

In [74]:
# Renommer les colonnes
taxE.columns = [
    'country',
    'ISO3',
    'year',
    'ECGTE_EnvTaxes_DomCCY',
    'ECGTE_EnvTaxes_PctGDP',
    'ECGTEN_TaxesEnergy_InclFuelTrans_DomCCY',
    'ECGTEN_TaxesEnergy_InclFuelTrans_PctGDP',
    'ECGTEP_TaxesPollution_DomCCY',
    'ECGTEP_TaxesPollution_PctGDP',
    'ECGTER_TaxesResources_DomCCY',
    'ECGTER_TaxesResources_PctGDP',
    'ECGTET_TaxesTrans_ExclFuelTrans_DomCCY',
    'ECGTET_TaxesTrans_ExclFuelTrans_PctGDP'
]
# Afficher les nouvelles colonnes
print(taxE.columns)

Index(['country', 'ISO3', 'year', 'ECGTE_EnvTaxes_DomCCY',
       'ECGTE_EnvTaxes_PctGDP', 'ECGTEN_TaxesEnergy_InclFuelTrans_DomCCY',
       'ECGTEN_TaxesEnergy_InclFuelTrans_PctGDP',
       'ECGTEP_TaxesPollution_DomCCY', 'ECGTEP_TaxesPollution_PctGDP',
       'ECGTER_TaxesResources_DomCCY', 'ECGTER_TaxesResources_PctGDP',
       'ECGTET_TaxesTrans_ExclFuelTrans_DomCCY',
       'ECGTET_TaxesTrans_ExclFuelTrans_PctGDP'],
      dtype='object')


In [75]:
# Compter le nombre de valeurs "NaN" dans les colonnes
nb_nan_colTax_taxE = taxE.isna().sum()

print("Nombre de NaN dans les colonnes :", nb_nan_colTax_taxE)

# Compter le nombre de valeurs "NaN" dans le dataset
nb_nan_dataset_taxE = taxE.isna().sum().sum()

print("Nombre de NaN dans le dataset :", nb_nan_dataset_taxE)

Nombre de NaN dans les colonnes : country                                      0
ISO3                                         0
year                                         0
ECGTE_EnvTaxes_DomCCY                      460
ECGTE_EnvTaxes_PctGDP                      477
ECGTEN_TaxesEnergy_InclFuelTrans_DomCCY    339
ECGTEN_TaxesEnergy_InclFuelTrans_PctGDP    345
ECGTEP_TaxesPollution_DomCCY               134
ECGTEP_TaxesPollution_PctGDP               140
ECGTER_TaxesResources_DomCCY               146
ECGTER_TaxesResources_PctGDP               152
ECGTET_TaxesTrans_ExclFuelTrans_DomCCY     350
ECGTET_TaxesTrans_ExclFuelTrans_PctGDP     367
dtype: int64
Nombre de NaN dans le dataset : 2910


In [76]:
# Filtrer les lignes avec des valeurs nan puis les afficher
nan_rows_taxE = taxE[taxE.isna().any(axis=1)]

print("Lignes contenant des valeurs NaN :")
print(nan_rows_taxE)

Lignes contenant des valeurs NaN :
                       country ISO3  year  ECGTE_EnvTaxes_DomCCY  \
5     Andorra, Principality of  AND  2015           4.600367e+12   
6     Andorra, Principality of  AND  2016           4.616889e+12   
7     Andorra, Principality of  AND  2017           5.043869e+12   
8     Andorra, Principality of  AND  2018           5.283535e+12   
9     Andorra, Principality of  AND  2019           5.675649e+12   
...                        ...  ...   ...                    ...   
2721                   Vietnam  VNM  1999                    NaN   
2722                   Vietnam  VNM  2000                    NaN   
2723                   Vietnam  VNM  2001                    NaN   
2724                   Vietnam  VNM  2002                    NaN   
2743                   Vietnam  VNM  2021                    NaN   

      ECGTE_EnvTaxes_PctGDP  ECGTEN_TaxesEnergy_InclFuelTrans_DomCCY  \
5                       NaN                                      0.0   
6   

In [77]:
taxE.dtypes

country                                     object
ISO3                                        object
year                                        object
ECGTE_EnvTaxes_DomCCY                      float64
ECGTE_EnvTaxes_PctGDP                      float64
ECGTEN_TaxesEnergy_InclFuelTrans_DomCCY    float64
ECGTEN_TaxesEnergy_InclFuelTrans_PctGDP    float64
ECGTEP_TaxesPollution_DomCCY               float64
ECGTEP_TaxesPollution_PctGDP               float64
ECGTER_TaxesResources_DomCCY               float64
ECGTER_TaxesResources_PctGDP               float64
ECGTET_TaxesTrans_ExclFuelTrans_DomCCY     float64
ECGTET_TaxesTrans_ExclFuelTrans_PctGDP     float64
dtype: object

In [78]:
taxE.shape

(2750, 13)

In [79]:
# Vérifier les valeurs nulles dans le DataFrame
null_values = taxE.isnull()

# Vérifier si le DataFrame est vide
is_empty = taxE.empty

# Afficher les valeurs nulles
print("Valeurs nulles dans le DataFrame :")
print(null_values)

# Afficher si le DataFrame est vide
print("\nLe DataFrame est vide :", is_empty)


Valeurs nulles dans le DataFrame :
      country   ISO3   year  ECGTE_EnvTaxes_DomCCY  ECGTE_EnvTaxes_PctGDP  \
0       False  False  False                  False                  False   
1       False  False  False                  False                  False   
2       False  False  False                  False                  False   
3       False  False  False                  False                  False   
4       False  False  False                  False                  False   
...       ...    ...    ...                    ...                    ...   
2745    False  False  False                  False                  False   
2746    False  False  False                  False                  False   
2747    False  False  False                  False                  False   
2748    False  False  False                  False                  False   
2749    False  False  False                  False                  False   

      ECGTEN_TaxesEnergy_InclFuelTrans_D

In [80]:
# (Titres anglais) Sélectionner les lignes avec NaN ou 0 dans les 10 colonnes spécifiées
lignes_nan_zero = taxE.loc[(taxE['ECGTE_EnvTaxes_DomCCY'].isna() | (taxE['ECGTE_EnvTaxes_DomCCY'] == 0)) & (taxE['ECGTE_EnvTaxes_PctGDP'].isna() | (taxE['ECGTE_EnvTaxes_PctGDP'] == 0)) &                                (taxE['ECGTEN_TaxesEnergy_InclFuelTrans_DomCCY'].isna() | (taxE['ECGTEN_TaxesEnergy_InclFuelTrans_DomCCY'] == 0)) & (taxE['ECGTEN_TaxesEnergy_InclFuelTrans_PctGDP'].isna() | (taxE['ECGTEN_TaxesEnergy_InclFuelTrans_PctGDP'] == 0)) & (taxE['ECGTEP_TaxesPollution_DomCCY'].isna() | (taxE['ECGTEP_TaxesPollution_DomCCY'] == 0)) & (taxE['ECGTEP_TaxesPollution_PctGDP'].isna() | (taxE['ECGTEP_TaxesPollution_PctGDP'] == 0)) & (taxE['ECGTER_TaxesResources_DomCCY'].isna() | (taxE['ECGTER_TaxesResources_DomCCY'] == 0)) & (taxE['ECGTER_TaxesResources_PctGDP'].isna() | (taxE['ECGTER_TaxesResources_PctGDP'] == 0)) & (taxE['ECGTET_TaxesTrans_ExclFuelTrans_DomCCY'].isna() | (taxE['ECGTET_TaxesTrans_ExclFuelTrans_DomCCY'] == 0)) & (taxE['ECGTET_TaxesTrans_ExclFuelTrans_PctGDP'].isna() | (taxE['ECGTET_TaxesTrans_ExclFuelTrans_PctGDP'] == 0))]

# Afficher les lignes sélectionnées
print(lignes_nan_zero)

         country ISO3  year  ECGTE_EnvTaxes_DomCCY  ECGTE_EnvTaxes_PctGDP  \
152   Bangladesh  BGD  1995                    NaN                    NaN   
153   Bangladesh  BGD  1996                    NaN                    NaN   
154   Bangladesh  BGD  1997                    NaN                    NaN   
155   Bangladesh  BGD  1998                    NaN                    NaN   
156   Bangladesh  BGD  1999                    NaN                    NaN   
...          ...  ...   ...                    ...                    ...   
2721     Vietnam  VNM  1999                    NaN                    NaN   
2722     Vietnam  VNM  2000                    NaN                    NaN   
2723     Vietnam  VNM  2001                    NaN                    NaN   
2724     Vietnam  VNM  2002                    NaN                    NaN   
2743     Vietnam  VNM  2021                    NaN                    NaN   

      ECGTEN_TaxesEnergy_InclFuelTrans_DomCCY  \
152                       

In [81]:
# (Titres anglais) Supprimer les lignes avec NaN ou 0 dans les 10 colonnes spécifiées
taxE = taxE.drop(taxE[(taxE['ECGTE_EnvTaxes_DomCCY'].isna() | (taxE['ECGTE_EnvTaxes_DomCCY'] == 0)) & (taxE['ECGTE_EnvTaxes_PctGDP'].isna() | (taxE['ECGTE_EnvTaxes_PctGDP'] == 0)) & (taxE['ECGTEN_TaxesEnergy_InclFuelTrans_DomCCY'].isna() | (taxE['ECGTEN_TaxesEnergy_InclFuelTrans_DomCCY'] == 0)) & (taxE['ECGTEN_TaxesEnergy_InclFuelTrans_PctGDP'].isna() | (taxE['ECGTEN_TaxesEnergy_InclFuelTrans_PctGDP'] == 0)) & (taxE['ECGTEP_TaxesPollution_DomCCY'].isna() | (taxE['ECGTEP_TaxesPollution_DomCCY'] == 0)) & (taxE['ECGTEP_TaxesPollution_PctGDP'].isna() | (taxE['ECGTEP_TaxesPollution_PctGDP'] == 0)) & (taxE['ECGTER_TaxesResources_DomCCY'].isna() | (taxE['ECGTER_TaxesResources_DomCCY'] == 0)) & (taxE['ECGTER_TaxesResources_PctGDP'].isna() | (taxE['ECGTER_TaxesResources_PctGDP'] == 0)) & (taxE['ECGTET_TaxesTrans_ExclFuelTrans_DomCCY'].isna() | (taxE['ECGTET_TaxesTrans_ExclFuelTrans_DomCCY'] == 0)) & (taxE['ECGTET_TaxesTrans_ExclFuelTrans_PctGDP'].isna() | (taxE['ECGTET_TaxesTrans_ExclFuelTrans_PctGDP'] == 0))].index)

# Réinitialiser les index après la suppression des lignes
taxE = taxE.reset_index(drop=True)

# Afficher le dataframe après suppression
taxE.head(5)

,country,ISO3,year,ECGTE_EnvTaxes_DomCCY,ECGTE_EnvTaxes_PctGDP,ECGTEN_TaxesEnergy_InclFuelTrans_DomCCY,ECGTEN_TaxesEnergy_InclFuelTrans_PctGDP,ECGTEP_TaxesPollution_DomCCY,ECGTEP_TaxesPollution_PctGDP,ECGTER_TaxesResources_DomCCY,ECGTER_TaxesResources_PctGDP,ECGTET_TaxesTrans_ExclFuelTrans_DomCCY,ECGTET_TaxesTrans_ExclFuelTrans_PctGDP
0,Albania,ALB,2015,4.399314e+10,3.067206,3.774111e+10,2.631314,1.782069e+09,0.124246,32546493.00,0.002269,4.437413e+09,0.309377
1,Albania,ALB,2016,4.781379e+10,3.247163,4.094562e+10,2.780726,1.879970e+09,0.127674,52524339.00,0.003567,4.935684e+09,0.335195
2,Albania,ALB,2017,4.754858e+10,3.066373,4.040004e+10,2.605369,1.941324e+09,0.125195,61861356.38,0.003989,5.145347e+09,0.331820
3,Albania,ALB,2018,5.114559e+10,3.124865,4.352182e+10,2.659072,2.226251e+09,0.136018,56011991.00,0.003422,5.341506e+09,0.326352
4,Albania,ALB,2019,5.341565e+10,3.157133,4.516530e+10,2.669496,2.625011e+09,0.155151,58777099.50,0.003474,5.566564e+09,0.329012


In [82]:
taxE = taxE.fillna(0)

In [83]:
taxE.to_csv('./output/Tax_Energy.csv', index=False)

In [84]:
# Changement de types
wec['year'] = wec['year'].astype(int)
data['year'] = data['year'].astype(int)
taxE['year'] = taxE['year'].astype(int)

wec['country'] = wec['country'].astype(str)
data['country'] = data['country'].astype(str)
taxE['country'] = taxE['country'].astype(str)

# Fusion des valeurs de Co2
wec = pd.merge(wec, taxE, on=["country", "year"], how='left')

# Valeurs enregistrés
wec.to_csv('./output/World_Energy_Consumption_Full.csv', index=False)

In [87]:
# Charger le fichier CSV dans un DataFrame
df_price = pd.read_csv('./assets/energy_prices.csv')

# Définir les nouveaux noms de colonnes
nouveaux_noms = {
    'Crude Oil': 'price_crude_oil',
    'Coal': 'price_coal',
    'Natural Gas': 'price_natural_gas',
    'Year': 'year'
}

# Renommer les colonnes
df_price.rename(columns=nouveaux_noms, inplace=True)

df_price = df_price.fillna(0)

df_price['price_crude_oil'] = df_price['price_crude_oil'].astype(float)
df_price['price_coal'] = df_price['price_coal'].astype(float)
df_price['price_natural_gas'] = df_price['price_natural_gas'].astype(float)
df_price['year'] = df_price['year'].astype(int)

df_price.head(10)



,year,price_crude_oil,price_coal,price_natural_gas
0,1960,1.63,0.0,0.14
1,1961,1.57,0.0,0.15
2,1962,1.52,0.0,0.16
3,1963,1.50,0.0,0.16
4,1964,1.45,0.0,0.15
5,1965,1.42,0.0,0.16
6,1966,1.36,0.0,0.16
7,1967,1.33,0.0,0.16
8,1968,1.32,0.0,0.16
9,1969,1.27,0.0,0.17


In [88]:
# Fusion des valeurs pour les prix des energies fossiles
wec = pd.merge(wec, df_price, on=["year"], how='left')
# Valeurs enregistrés
wec.to_csv('./output/World_Energy_Consumption_Full.csv', index=False)


In [89]:
# Charger le dataset World_Energy_Consumption_Full.csv
price = pd.read_csv('./output/World_Energy_Consumption_Full.csv')
price.head(10)

,country,year,iso_code,population,gdp,biofuel_cons_change_pct,biofuel_cons_change_twh,biofuel_cons_per_capita,biofuel_consumption,biofuel_elec_per_capita,...,ECGTEN_TaxesEnergy_InclFuelTrans_PctGDP,ECGTEP_TaxesPollution_DomCCY,ECGTEP_TaxesPollution_PctGDP,ECGTER_TaxesResources_DomCCY,ECGTER_TaxesResources_PctGDP,ECGTET_TaxesTrans_ExclFuelTrans_DomCCY,ECGTET_TaxesTrans_ExclFuelTrans_PctGDP,price_crude_oil,price_coal,price_natural_gas
0,Afghanistan,1961,AFG,8790140.0,1.314629e+10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.57,0.0,0.15
1,Afghanistan,1962,AFG,8969055.0,1.336763e+10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.52,0.0,0.16
2,Afghanistan,1963,AFG,9157463.0,1.363030e+10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.50,0.0,0.16
3,Afghanistan,1964,AFG,9355510.0,1.387050e+10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.45,0.0,0.15
4,Afghanistan,1965,AFG,9565154.0,1.418742e+10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.42,0.0,0.16
5,Afghanistan,1966,AFG,9783153.0,1.432526e+10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.36,0.0,0.16
6,Afghanistan,1967,AFG,10010037.0,1.473403e+10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.33,0.0,0.16
7,Afghanistan,1968,AFG,10247782.0,1.525425e+10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.32,0.0,0.16
8,Afghanistan,1969,AFG,10494491.0,1.549319e+10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.27,0.0,0.17
9,Afghanistan,1970,AFG,10752973.0,1.581223e+10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.21,7.8,0.17
